# Telegram bot
# ![](images/telegram.jpg)


#### Существует достаточно много различных библиотек для создания телеграм ботов, например:
***
- AIOGram - это довольно простой и полностью **асинхронный** фреймворк для Telegram Bot API, написанный на Python 3.7
  
  Документацию к данной библиотеке можно найти тут: https://docs.aiogram.dev/en/latest/
  
  Ссылка на репозиторий библиотеки https://github.com/aiogram/aiogram
  
***
- python-telegram-bot - пакет python-telegram-bot предоставляет чистый интерфейс Python для Telegram Bot API. Он совместим с версиями Python 3.6+. В дополнение к чистой реализации API эта библиотека содержит ряд высокоуровневых классов, которые делают разработку ботов простой и понятной. Эти классы содержатся в модуле telegram.ext.

    Ссылка на репозиторий библиотеки https://github.com/python-telegram-bot/python-telegram-bot
    
***
    
- Telepot - работает на Python 2.7 и Python 3. Для Python 3.5+ он также имеет **асинхронную** версию, основанную на asyncio.

    Ссылка на репозиторий библиотеки https://github.com/nickoala/telepot

***
- telebot - хорошая и лёгкая библиотека для создания бота на python для telegram.
    
    Ссылка на репозиторий библиотеки https://github.com/tucnak/telebot
    
***    
- twx.botapi - это python-интерфейс для Telegram bot API. Он поддерживает выполнение **синхронных** и **асинхронных** вызовов и преобразует ответ в пригодный для использования собственный объект python.

    Ссылка на репозиторий библиотеки https://github.com/datamachine/twx.botapi
    
    Python version supported 2.7, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6

    
***
- pyTelegramBotAPI - простая, но расширяемая реализация на Python для Telegram Bot API. Может быть как **синхронный**, так и **асинхронный**.Это мощный инструмент для вообще чего угодно. Автоматизация действий, работа с пользователями, онлайн-магазины, игры и тд. Поддержка Python 3.6-3.10

    Ссылка на репозиторий библиотеки https://github.com/eternnoir/pyTelegramBotAPI
    
***
#### В чем различия асинхронного бота от обычного?

Представим что пользователи шлют через специальные клиенты telegram боту сообщение. Наш бот получает сообщение и приступает к выполнению заранее прописанной логики, которая как-то зависит от сообщения. На этом этапе telebot и например aiogram ничем не отличаются.

Предположим, что нам потребовалось обратиться в БД для получения какой-то информации, что мы можем сделать?

Послать запрос и просто ждать ответа. Тем временем очередь из входящих сообщений может копиться.
Или же послать запрос и обрабатывать другое сообщение, но вернуться к этому позже, когда результат будет готов.
В первом случае сообщение обрабатывается синхронно, во втором асинхронно. В этом проявляются ключевые различия.

***
Библиотеки значительно упрощают написание ботов в telegram, но так же мы можем написать бота и без использования готовых библиотек [Пример](https://habr.com/ru/company/kts/blog/598575)
    

### Polling и long polling
Чат-боты должны получать уведомления от соцсети моментально. Они не могут проверять обновления каждую секунду, это неэффективно. Большинство ответов от сервера будут неинформативны, по типу "Новых сообщений нет".

Такой подход, когда раз в n секунд опрашивается сторонний сервис, называется **polling**.

# ![](images/polling.png)

Чтобы сэкономить на ресурсах, можно использовать long polling. Он успроен так же, как и polling, с одним отличием: сервер дольше отвечает. При long polling сервер отвечает в двух случаях: или потому, что пришло новое сообщение, или потому, что соединение пора разрывать.

У каждого запроса есть timeout(время, в течении которого нужно ответить). Если на запрос не ответили за это время, считается, что сервер не ответит вообще. Поэтому сервер смотрит на timeout и решает так:

Если за это время не появится обновлений для клиента, я отвечу ему, что их нет.
Если появятся, я отправлю ему обновления сразу, не дожидаясь таймаута.
Чтобы реализовать long polling на стороне клиента, нужно выставить большой timeout: 30 или 60 секунд.

# ![](images/long_polling.png)
 
***
Вот так выглядит polling со стороны клиента на Python:

```python
from time import sleep
import requests

while True:
    response = requests.get("http://someurl.com")
    for message in response:
        bot.answer(message)
        sleep(1)
```
***
А вот так выглядит long polling:

```python
import requests

while True:
    response = requests.get("http://someurl.com", timeout=60)
    for message in response:
        bot.answer(message)
```

### Webhook

Это понятие тесно связано с предыдущим

**Polling** — это когда клиент всё время спришвает у сервера “есть что-нибудь новенькое?”, а сервер присылает события (например, новые сообщения) или пишет, что ничего не происходило. 
**Webhook** — это то же самое, только наоборот. Теперь если что-то случается (например, новое сообщение) — сервер сам сообщит об этом клиенту

# ![](images/webhook.png)

Рассмотрим эту схему чуть подробнее. Есть 2 основных действующих лица: клиент и сервер уведомлений. Как только вашему боту написал пользователь — сервер уведомлений шлёт вам JSON-объект, где указывает кто писал, что писал и тдругую информацию. Дальше клиент сам решает, как на это сообщение отреагировать. Он может прислать одно сообщение, может несколько, а может вообще промолчать. В конце он отвечает серверу “Ок, 200”(это значит что запрос обработан)

Чтобы клиент мог писать в ответ, обычно создаётся отдельное API, где реализованы запросы на отправку сообщений и так далее. API и сервер уведомлений — это обычно одна та же соцсеть, просто разные микрсервисы. Один занимается только рассылкой уведомлений, другой — полноценное API для разработчиков.

#### webhook или polling?
Обе технологии занимаются одним и тем же: приносят вам сообщения о событиях. Тем не менее, между ними огромная разница: polling легко реализуется обычной библиотекой requests, в то время как для webhook обязательно нужен сайт, который будет принимать и обрабатывать запросы от сервера уведомлений.

Взамен webhook снижает нагрузку на сеть: вы обмениваетесь сообщениями с сервером уведомлений только “по делу”, в то время как polling постоянно отправляет запрос на наличие новых сообщений.

При этом вы не теряете мгновенность ответа, как при long polling, потому что сервер по-прежнему пишет вам сразу, как только ему написал пользователь.

#### Теперь попробуем реализовать телеграм бота с помощью библиотеки aiogram

Для начала ее необходимо установить

In [9]:
pip install -U aiogram

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.3/196.3 KB 1.0 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.2/160.2 KB 966.5 kB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 873.4 kB/s eta 0:00:00eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 2.7 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 KB 1.4 MB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.0/264.0 KB 1.8 MB/s eta 0:00:003.1 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 KB 1.6 MB/s eta 0:00:00 MB/s eta 0:00:01:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note:

В качестве примера создадим телеграмм бота который будет приветствовать пользователя и в качестве ответа присылать ему полученное сообщение.

Для начала создадим бота, для этого в поиске телеграмма находим **@BotFather**

Теперь вводим команду /newbot. Затем вводим имя и юзернейм. Обратите внимание, что юзернейм должен оканчиваться на bot!

# ![](images/example_1)

Мы получили токен(его лучше хранить в отдельном файле, например congfig.py)

In [1]:
TOKEN = "5693758372:AAFn1OcVV5I-JWGW_hycqV-6Inoja6nHdKQ"

Что бы запустить нашего бота из jupyter-notebook выполним:

In [2]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

Defaulting to user installation because normal site-packages is not writeable


Теперь приступаем к редактированию файла bot.py. Для этого импортируем необходимые модули библиотеки aiogram и токен бота, а так же инициализируем объекты бота и диспетчера, а также импортируем токен из config.py
```python
    from config import TOKEN
```

In [3]:
from aiogram import Bot, types
from aiogram.dispatcher import Dispatcher
from aiogram.utils import executor



bot = Bot(token=TOKEN)
dp = Dispatcher(bot)

Команда, с которой начинается общение пользователя с ботом - **/start**. Поэтому давайте научим нашего бота реагировать на эту команду. Создаем message_handler и объявляем там функцию ответа

In [4]:
@dp.message_handler(commands=['start'])
async def process_start_command(message: types.Message):
    await message.reply("Привет!\n")

Ещё в ботах принято создавать обработчик команды **/help**- вдруг пользователь заинтересуется возможностями бота.
Создадим отдельный message_handler для этой команды

In [5]:
@dp.message_handler(commands=['help'])
async def process_help_command(message: types.Message):
    await message.reply("Напиши мне что-нибудь, и я повторю за тобой!")

Осталось сделать обработку текстового сообщения. Для этого пишем следующее:

In [6]:
@dp.message_handler()
async def echo_message(msg: types.Message):
    await bot.send_message(msg.from_user.id, msg.text)

Чтобы получать сообщения от серверов Telegram воспользуемся polling (постоянным опросом сервера на наличие новых обновлений)

In [ ]:
# import asyncio
executor.start_polling(dp)


# ![](images/example_2)
